# Power

## Power pins
- AVDDH(3.3 / 2.5 V) [2 x pins]
- AVDDL (1.2 V) [2 x pins]
- DVDLL (1.2 V) [VDD] [4 x pins]
- DVDDH (3.3, 2.5 or 1.8 V) [VDD] [2 x pins]
- VSS (GND)
- LDO_O (Contains control to drive FET for 1.2 V output)
	- Leave unused, use your own 1.2 V supply
- AVDD_PL (1.2 V for PLL)

### 1.2 V power supply
- Add an extra buck converter generating 1.2 V

In [2]:
import numpy as np

def get_resistor_ratio(v_out):
	return ((v_out / 0.6) - 1)

resistor_array = np.array([20, 22, 24, 27, 30, 33, 39, 47, 51, 56, 68, 75, 82, 100, 150])
vdd_targets = np.array([1.2])
# r1_arr = [30, 75, 100]
# r2_arr = [24, 24, 22]

######### DDR3L
for idx, v_target in enumerate(vdd_targets):
	print(f"---- V = {v_target} ----")
	v_target_ratio = get_resistor_ratio(v_target)
	r2_target_arr = np.round(resistor_array / v_target_ratio, 2)
	r1_target_arr = np.round(resistor_array * v_target_ratio, 2)
	print(f"{v_target} r1/r2: {v_target_ratio}")
	print(f"R2 values {r2_target_arr}")
	print(f"R1 values {r1_target_arr}")

	# for r1, r2 in zip(r1_target_arr, r2_target_arr):
	# 	if (r1 in resistor_array):
	# 		print(f"Perfect r1 found: {r1}")
	# 	if (r2 in resistor_array):
	# 		print(f"Perfect r2 found: {r2}")

	# r1 = r1_arr[idx]
	# r2 = r2_arr[idx]
	# vout_actual = 0.6*(1+r1/r2)
	# print(f"offset: {((vout_actual - v_target) / v_target) * 100:.2f}%")

---- V = 1.2 ----
1.2 r1/r2: 1.0
R2 values [ 20.  22.  24.  27.  30.  33.  39.  47.  51.  56.  68.  75.  82. 100.
 150.]
R1 values [ 20.  22.  24.  27.  30.  33.  39.  47.  51.  56.  68.  75.  82. 100.
 150.]


**Chose 2 x 100 kOhm**

## Decoupling capacitors / Ferrites

Reference: Check page 9, ANLAN206 – KSZ9031 G

- Ferrites: 
	- < 0.1 ohms in DC Resistance
	- 60 ohm impedance at 100 MHz
	- example: MPZ1005S600C
- Capacitors
	- X7R / X5R
	- Mostly HF decoupling (Do bulk at source)

### AVDD_PLL (1.2 V)
- Ferrite
- 10 uF bulk
- 100 nF decoupling

### AVDDL (1.2)
- Ferrite
- 10 uF bulk
- 100 nF per pin


### AVDDH (3.3 V)
- Ferrite
- 10 uF X5R bulk capacitance
- 1 uF X7R bulk capacitance
- 0.1 uF decoupling per pin

### DVDDL (1.2 V)
- 100 nF decoupling on each pin
- Bulk capacitance close enough (10 uF)

### DVDDH (3.3 V)
- 100 nF per pin
- 10 uF bulk

## Notes
### Filtering with ferrites vs filtering with inductors
Ferrites
- Poorly characterized
- Dissipate resistively (also at DC!)
- Cheap
- Don't resonate

Inductors
- Well-characterized (up to a certain frequency)
	- 20 dB/dec drop, sharp resonance-filters possible here.
- Impedance peak: $f_{0}=\frac{1}{\sqrt{2*\pi*L*C}}$

# Transmit / Receive signals RGMMI
- Use Bank 1 for these signals
- We don't need any DQS groups for a RGMII interface
	- Rise/Fall/Delay time of a regular FPGA PIO:
		- LVCMOS33 has a max output frequency of 150 MHz

## Transmit / Receive
- TXDi
- RXDi
### RXDi latching
- RXDi, RX_CLK, RX_DV
Pull-ups, pull-downs are latched to select the relevant mode during power-on / RESET



# Clock

- XO/XI (clock input / output with crystal feedback)
### MDIO / MDC
- Management Data Clock Input
- Management data input / Output
	- Pull-up (1-4.7 kOhms to VDD_IO)
